<a href="https://colab.research.google.com/github/Rashi-Dwivedi1812/Rag-Query-Engine/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyMuPDF -q
!pip install langchain_community
!pip install sentence-transformers
!pip install langchain langchain-community sentence-transformers faiss-cpu PyMuPDF
!pip install langchain langchain-community sentence-transformers faiss-cpu PyMuPDF groq langchain-groq python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.8 MB/s eta 0:00:00


In [2]:
import os

# Define the source directory for your raw files
source_directory = "raw_data"

# Define the destination directory for the converted text files
destination_directory = "processed_data"

# Create the directories
os.makedirs(source_directory, exist_ok=True)
os.makedirs(destination_directory, exist_ok=True)

print(f"Directory '{source_directory}' created.")
print(f"Please upload your PDF and JSON files to the '{source_directory}' folder in the file browser on the left.")

Directory 'raw_data' created.
Please upload your PDF and JSON files to the 'raw_data' folder in the file browser on the left.


In [3]:
import os
import json
import fitz  # PyMuPDF

# --- Function to Convert PDFs to Text ---
def convert_pdfs_to_txt(source_dir: str, dest_dir: str):
    """Converts all PDF files in a source directory to text files."""
    print("--- Converting PDF files to text... ---")
    if not os.path.exists(source_dir):
        print(f"Source directory '{source_dir}' not found.")
        return

    for filename in os.listdir(source_dir):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(source_dir, filename)
            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_path = os.path.join(dest_dir, txt_filename)

            try:
                doc = fitz.open(pdf_path)
                text_content = ""
                for page in doc:
                    text_content += page.get_text()

                with open(txt_path, "w", encoding="utf-8") as f:
                    f.write(text_content)

                print(f"Successfully converted '{filename}' to '{txt_filename}'.")
            except Exception as e:
                print(f"Error converting '{filename}': {e}")


# --- Function to Convert JSON to Text ---
def convert_json_to_txt(source_dir: str, dest_dir: str):
    """Converts all JSON files with Q&A pairs to a single text file."""
    print("\n--- Converting JSON files to text... ---")
    if not os.path.exists(source_dir):
        print(f"Source directory '{source_dir}' not found.")
        return

    for filename in os.listdir(source_dir):
        if filename.lower().endswith(".json"):
            json_path = os.path.join(source_dir, filename)
            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_path = os.path.join(dest_dir, txt_filename)

            try:
                with open(json_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                text_content = ""
                # Check if data is a list of dictionaries (Q&A format)
                if isinstance(data, list):
                    for entry in data:
                        if "question" in entry and "answer" in entry:
                            text_content += f"Question: {entry['question']}\nAnswer: {entry['answer']}\n\n"
                else:
                    print(f"Warning: JSON file '{filename}' is not in the expected Q&A list format. Skipping.")
                    continue


                with open(txt_path, "w", encoding="utf-8") as f:
                    f.write(text_content)

                print(f"Successfully converted '{filename}' to '{txt_filename}'.")
            except Exception as e:
                print(f"Error converting '{filename}': {e}")


# --- Main Execution Block ---

# Define directory paths (using the variables from the previous cell)
source_data_dir = "raw_data"
dest_data_dir = "processed_data"

# Run the conversion functions
convert_pdfs_to_txt(source_data_dir, dest_data_dir)
convert_json_to_txt(source_data_dir, dest_data_dir)

print(f"\nConversion complete! All processed documents are now in the '{dest_data_dir}' directory.")
print("You can now use these text files for your next processing steps.")

--- Converting PDF files to text... ---
Successfully converted 'mql5book.pdf' to 'mql5book.txt'.

--- Converting JSON files to text... ---

Conversion complete! All processed documents are now in the 'processed_data' directory.
You can now use these text files for your next processing steps.


In [4]:
import os
import re # Import the regular expressions library
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document # Import Document class

# Define paths
# Make sure this path points to the text files generated by your first script
DOCS_PATH = "processed_data"
FAISS_INDEX_PATH = "data/faiss_index"

def separate_code_and_text(content: str):
    """
    Separates text content into natural language and code blocks
    using markdown-style ``` fences.
    """
    # This regex finds all blocks enclosed in ``` ... ```
    code_pattern = re.compile(r"```(.*?)```", re.DOTALL)

    code_blocks = []
    # Find all matches and store them
    for match in code_pattern.finditer(content):
        # The content of the code block is in group 1
        code_blocks.append(match.group(1).strip())

    # Remove the found code blocks from the original content to get the text
    text_content = code_pattern.sub("", content).strip()

    return text_content, code_blocks


def create_faiss_index():
    """
    Processes documents, separates code from text, creates embeddings with metadata,
    and saves a FAISS index.
    """
    print("Starting the FAISS index creation process...")

    all_docs = [] # We will store LangChain Document objects here

    if not os.path.exists(DOCS_PATH):
        print(f"Error: Directory '{DOCS_PATH}' not found.")
        return

    # 1. Load each file and process its content
    for filename in os.listdir(DOCS_PATH):
        if filename.endswith(".txt"):
            file_path = os.path.join(DOCS_PATH, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

            # Separate text and code
            text, code = separate_code_and_text(content)

            # Create Document objects with metadata
            if text:
                all_docs.append(Document(page_content=text, metadata={"source": filename, "type": "text"}))

            for i, code_block in enumerate(code):
                if code_block:
                    # Give each code block a unique source identifier
                    code_source = f"{filename}_code_{i+1}"
                    all_docs.append(Document(page_content=code_block, metadata={"source": code_source, "type": "code"}))

    if not all_docs:
        print(f"No processable content found in '{DOCS_PATH}'.")
        return

    print(f"Loaded and processed {len(all_docs)} content blocks (text and code). Splitting...")

    # 2. Split documents into smaller chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100) # Increased overlap
    split_docs = text_splitter.split_documents(all_docs)

    print(f"Split into {len(split_docs)} chunks. Generating embeddings...")

    # 3. Create embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    # 4. Create the FAISS index from documents with metadata
    db = FAISS.from_documents(split_docs, embeddings)
    db.save_local(FAISS_INDEX_PATH)

    print("FAISS index created successfully!")
    print(f"Index saved at: {FAISS_INDEX_PATH}")


if __name__ == "__main__":
    # Create the destination directory if it doesn't exist for the index
    os.makedirs(os.path.dirname(FAISS_INDEX_PATH), exist_ok=True)
    create_faiss_index()

Starting the FAISS index creation process...
Loaded and processed 1 content blocks (text and code). Splitting...
Split into 1 chunks. Generating embeddings...


/tmp/ipython-input-3675513209.py:80: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index created successfully!
Index saved at: data/faiss_index


In [7]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import Document
from dotenv import load_dotenv

# --- CONFIGURATION ---
CONFIG = {
    "FAISS_INDEX_PATH": "data/faiss_index",
    "EMBEDDING_MODEL_NAME": "sentence-transformers/all-MiniLM-L6-v2",
    "LLM_MODEL_NAME": "llama-3.1-8b-instant",
    "SEARCH_K": 3,  # Retrieve a few docs to have options for trimming
    "MAX_CONTEXT_CHARS": 4000  # Safely under the model's context limit
}

# --- STEP 1: API KEY SETUP ---
def setup_api_key():
    """
    Sets up the Groq API key from Colab secrets or a local .env file.
    """
    try:
        from google.colab import userdata
        os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    except ImportError:
        load_dotenv()

    if not os.getenv("GROQ_API_KEY"):
        raise ValueError("GROQ_API_KEY not found. Please set it in Colab secrets or a .env file.")

# --- STEP 2: CONTEXT TRIMMER ---
def trim_context(docs, max_chars=CONFIG["MAX_CONTEXT_CHARS"]):
    """
    Safely trims a list of documents to ensure the total character count
    is within the specified limit.
    """
    total_chars = 0
    trimmed_docs = []
    for doc in docs:
        doc_len = len(doc.page_content)
        if total_chars + doc_len <= max_chars:
            trimmed_docs.append(doc)
            total_chars += doc_len
        else:
            # Stop adding documents if we exceed the limit
            break
    return trimmed_docs

# --- STEP 3: RAG SETUP ---
def setup_rag_components():
    """
    Loads all necessary components for the RAG pipeline.
    Returns the document chain and the retriever separately.
    """
    print("--- Setting up RAG Components ---")

    # 1. Load LLM
    print(f"Loading LLM: {CONFIG['LLM_MODEL_NAME']} via Groq...")
    llm = ChatGroq(model_name=CONFIG['LLM_MODEL_NAME'], temperature=0.1)

    # 2. Load Embeddings
    print(f"Loading Embedding Model: {CONFIG['EMBEDDING_MODEL_NAME']}...")
    embeddings = HuggingFaceEmbeddings(model_name=CONFIG['EMBEDDING_MODEL_NAME'])

    # 3. Load FAISS Index
    if not os.path.exists(CONFIG['FAISS_INDEX_PATH']):
        raise FileNotFoundError(
            f"FAISS index not found at '{CONFIG['FAISS_INDEX_PATH']}'. Run the indexing script first."
        )
    print(f"Loading FAISS index from: {CONFIG['FAISS_INDEX_PATH']}...")
    db = FAISS.load_local(
        CONFIG['FAISS_INDEX_PATH'],
        embeddings,
        allow_dangerous_deserialization=True
    )

    # 4. Create Retriever
    retriever = db.as_retriever(search_kwargs={"k": CONFIG['SEARCH_K']})

    # 5. Prompt Template
    prompt_template = """
You are a precise technical assistant. Answer ONLY using the context provided.

If the question asks for code:
1. Show the full code block from context.
2. Then explain it briefly.

If the answer isn’t in context, reply: "Information not available."

CONTEXT:
{context}

QUESTION:
{input}

ANSWER:
"""
    prompt = PromptTemplate.from_template(prompt_template)

    # 6. Create the Document Chain (which stuffs the prompt and calls the LLM)
    document_chain = create_stuff_documents_chain(llm, prompt)

    print("--- RAG Components setup complete. Ready for questions. ---")
    return document_chain, retriever

# --- STEP 4: MAIN LOOP ---
def main():
    try:
        setup_api_key()
        # Get the components needed for our manual workflow
        document_chain, retriever = setup_rag_components()

        while True:
            print("\n" + "=" * 50)
            user_query = input("Ask a question about your documents (or type 'exit' to quit): ")
            if user_query.lower() in ["exit", "quit"]:
                print("Exiting application. Goodbye!")
                break
            if not user_query.strip():
                continue

            # 1. Retrieve relevant documents
            print("Retrieving documents...")
            retrieved_docs = retriever.invoke(user_query)

            # 2. Trim the context to prevent overflow
            print(f"Trimming context to fit under {CONFIG['MAX_CONTEXT_CHARS']} characters...")
            trimmed_docs = trim_context(retrieved_docs)

            # 3. Invoke the document chain directly with the trimmed context
            print("Generating answer...")
            response = document_chain.invoke({"input": user_query, "context": trimmed_docs})

            print("\n--- Answer ---")
            print(response)

            # 4. Show the sources that were actually used
            print("\n--- Sources Used ---")
            if trimmed_docs:
                sources = sorted({doc.metadata.get("source", "Unknown") for doc in trimmed_docs})
                for src in sources:
                    print(f"- {src}")
            else:
                print("No sources were retrieved from the knowledge base.")

    except FileNotFoundError as e:
        print(f"\n[ERROR] {e}")
    except ValueError as e:
        print(f"\n[ERROR] {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

# --- RUN APP ---
if __name__ == "__main__":
    main()



--- Setting up RAG Components ---
Loading LLM: llama-3.1-8b-instant via Groq...
Loading Embedding Model: sentence-transformers/all-MiniLM-L6-v2...
Loading FAISS index from: data/faiss_index...
--- RAG Components setup complete. Ready for questions. ---

Ask a question about your documents (or type 'exit' to quit): why  MQL5 allows saving the results in files and databases
Retrieving documents...
Trimming context to fit under 4000 characters...
Generating answer...

--- Answer ---
MQL5 allows saving the results in files and databases because it provides a set of functions for file and database operations. These functions enable developers to read and write data to various file formats, such as CSV, XML, and binary files, as well as interact with databases like MySQL and SQLite.

Here's an example of how to save data to a file using MQL5:

```c
// Save data to a file
void SaveDataToFile(string filename, string data)
{
   // Open the file for writing
   FileOpen(filename, FILE_WRITE);
   